# Setup

In [ ]:
# ! pip install langchain langchain-community faiss-cpu sentence-transformers octoai-sdk langchain-text-splitters lxml tiktoken python-dotenv 'arize-phoenix[evals]' openai

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
import phoenix as px
session = px.launch_app()

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

# Ingest Data

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

In [ ]:
files = os.listdir("./city_data")
file_texts = []
for file in files:
    with open(f"./city_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=800, chunk_overlap=200, separator="."
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

WARNI [langchain_text_splitters.base] Created a chunk of size 1311, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 1043, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 1110, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 925, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 831, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 990, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 908, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 992, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Created a chunk of size 928, which is longer than the specified 800
WARNI [langchain_text_splitters.base] Creat

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
embeddings = HuggingFaceEmbeddings()

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
vector_store = FAISS.from_documents(
    file_texts,
    embedding=embeddings
)

In [ ]:
len(file_texts)

541

# Search the Data

In [ ]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="mistral-7b-instruct-v0.3",
        max_tokens=2000,
        presence_penalty=0,
        temperature=0.0,
        top_p=0.9,
    )

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate
template="""You are a tour guide. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("Where is the best food in the world?")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


' Berlin is known for its diverse gastronomy scene, with a wide range of international cuisines available, including Turkish, Arab, Asian, and Italian. The city is also home to many traditional German restaurants, serving dishes such as currywurst, schnitzel, and sauerbraten. Berlin has a vibrant food culture, with many street food markets, supper clubs, and food festivals throughout the year. The city is also famous for its traditional German bakeries, offering a variety of breads and pastries. Overall, Berlin is a great destination for foodies, with a wide range of options to suit all tastes and budgets.'

# Run Evals

In [ ]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

In [ ]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

In [ ]:
eval_model = OpenAIModel(
    model="gpt-4-turbo",
)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)

WARNI [phoenix.evals.executors] 🐌!! If running llm_classify inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
run_evals |██████████| 10/10 (100.0%) | ⏳ 01:21<00:00 |  8.10s/it
WARNI [phoenix.evals.executors] 🐌!! If running llm_classify inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
run_evals |██████████| 20/20 (100.0%) | ⏳ 02:53<00:00 |  8.68s/it
